## Saldo y cantidad de tarjetas en el sistema financiero.
Fuente: https://www.bcp.gov.py/web/institucional/boletines-estad%C3%ADstico-financieros-

In [ ]:
## Proceso par al carga de Market share de Tarjetas BCP
# Configuración logging
# Primero, configuramos el logger al inicio del script:
import os
import logging
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import logging
import pyodbc

# Obtener la ruta actual
ruta_actual = os.getcwd()
# Configurar logging
logging.basicConfig(
    filename=f'{ruta_actual}/etl_proceso.log',       # Archivo de log
    level=logging.INFO,               # Nivel de log: DEBUG, INFO, WARNING, ERROR, CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'                      # 'w' sobrescribe cada vez, 'a' acumula
)

logger = logging.getLogger()
print('Se inicializa parametros')

In [ ]:
## ruta del archivo
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'

def extraer_excel(excel_file):
    try:
        logger.info(f"Iniciando extracción desde: {excel_file}")
        sheets_dict = pd.read_excel(excel_file, sheet_name=None)
        # Acceder a cada hoja como DataFrame
        for sheet_name, df in sheets_dict.items():
            print(f"Hoja: {sheet_name}, Filas: {len(df)}")
        # logger.info(f"Iniciando extracción desde: {excel_file}")
        # sheets_dict = pd.read_excel(excel_file, sheet_name=None)
        # # Acceder a cada hoja como DataFrame
        # for sheet_name, df in sheets_dict.items():
        #     print(f"Hoja: {sheet_name}, Filas: {len(df)}")
        #     logger.info(f"Hoja: {sheet_name}, Filas: {len(df)}")
        return sheets_dict
    except Exception as e:
        logger.error(f"Error en la extracción: {e}")
        raise

In [ ]:
# Obtener ruta actual
ruta_actual = os.getcwd()
logger.info(f"Ruta actual de ejecución: {ruta_actual}")

# Ruta del archivo Excel
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'

# Extracción de hojas
try:
    logger.info(f"Iniciando lectura del archivo Excel: {excel_file}")
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    logger.info(f"Lectura exitosa. Hojas encontradas: {list(sheets_dict.keys())}")
    
    # Acceder a cada hoja como DataFrame
    for sheet_name, df in sheets_dict.items():
        logger.info(f"Hoja: {sheet_name}, Filas: {len(df)}")
        print(f"Hoja: {sheet_name}, Filas: {len(df)}")

except Exception as e:
    logger.error(f"Error al leer el archivo Excel: {e}")
    print(f"Error: {e}")

In [ ]:
## Vista previa de los datos la hoja
sheets_dict.get('Carteras').head()

In [ ]:
## version  sin Looger
def transforBol(df):
    # Eliminar filas vacías
    df = df.dropna(how='all')
    
    # Renombrar columnas
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    
    # Convertir fechas
    if 'fecha' in df.columns:
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
    
    return df

# Aplicar transformación a cada hoja
datos_transformados = {sheet: transforBol(df) for sheet, df in sheets_dict.items()}

In [ ]:
datos_transformados.get('Credito Sector').head(3)

In [ ]:
# Parámetros de conexión
server = 'ASUSTUF\SQL22'  # Doble barra para escapar correctamente
database = 'BolBcp'

try:
    logger.info(f"Intentando conectar a SQL Server: {server}, Base de datos: {database}")
    
    connP = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )
    
    logger.info("Conexión a SQL Server establecida exitosamente.")
    print('Conexión exitosa')

except pyodbc.Error as e:
    logger.error(f"Error al conectar a la base de datos: {str(e)}")
    print(f'Error al conectar a la base de datos: {str(e)}')

In [ ]:
# Cargar cada hoja transformada como tabla
for nombre_tabla, df in datos_transformados.items():
    try:
        logger.info(f"Iniciando carga de hoja: {nombre_tabla}")

        # Eliminar filas completamente vacías
        df = df.dropna(how='all')
        # Agregar columna de fecha y hora de carga
        df['fecha_carga'] = datetime.now()

        # Insertar usando pandas
        df.to_sql(name=nombre_tabla, con=engine, if_exists='replace', index=False)

        logger.info(f"Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")
        print(f"✅ Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")

    except Exception as e:
        logger.error(f"Error al cargar hoja '{nombre_tabla}': {e}")
        print(f"❌ Error al cargar hoja '{nombre_tabla}': {e}")